# IMDb Movies: Gross Income Prediction for Box Office Earnings

# Names


- Carly Freedman
- Jackson Teel
- Ye Yint Win
- Garrett Dungca

# Abstract 

We would like to create a program to predict the gross income in U.S. dollars of movies based on their year, certificate, runtime, genre, rating and total number of votes. The certificate gives the movie rating (i.e. PG-13), the rating represents the total score received by the movies from reviewers on IMDB, and the total number of votes is how many people reviewed the movie total. We will be using a dataset containing all of these variables for every movie on IMDB. We are going to one-hot encode each of the categories. In order to predict the gross income we will be using and comparing several regression models such as decision tree regression, LASSO, Ridge, and linear regression. We will use a loss value based on the actual gross income vs. predicted gross income.

# Background

There have been many studies conducted on predicting gross income of movies. However, there are few accurate ones. Most studies produce an accuracy in the 50-60% range. For example, a study conducted on this topic using both standard linear regression and classification via logistic regression only produced an accuracy of about 75% <a name="yoo"></a>[<sup>[1]</sup>](#yoonote). Given that this study was likely limited by the types of analysis they performed, we would like to determine if there is any way to increase this accuracy using a wider variety of algorithms and a different dataset. In our own analysis, we will likely use standard linear regression as well, but as a baseline reference for our other algorithms.

Using machine learning algorithms to predict gross income of movies has several benefits. Accurate income predictions can help movie studios, producers, and investors make better business decisions. It allows for more informed decisions to be made with regard to budgeting, marketing, and distribution<a name="dhir"></a>[<sup>[2]</sup>](#dhirnote). These predictions can also help filmmakers and studios create movies that are more engaging to their audience. ML algorithms can be used to analyze audience preferences based on demographics and other key variables that help inform creative decisions. From the article titled, ‘Popularity prediction of movies: from statistical modeling to machine learning techniques’, we were inspired to use a Decision Tree model for our predictions as it turned out to be one of the more accurate models in their case. <a name="abidi"></a>[<sup>[3]</sup>](#abidinote).

# Problem Statement
Our problem is how to predict the gross box office earnings of a movie based on features such as genre, rating, runtime, certificate, year, and votes. The metric we aim to optimize is the loss of the amount in US dollars of gross box office earnings compared with the predicted values, using IMDB's labeled dataset of movies. Our specific loss is undetermined at the moment, it is more likely that we will explore a variety of losses given the wide range of earnings a movie can make. This includes but is not limited to:
MAPE: Good for movies earning greater amounts even if the prediction is off, worse for predictions with low earning movies (proportion is greater with smaller numbers)
MSE: Useful but difficult to interpret especially with heavy mispredictions (could choose to look at earnings in # of millions of dollars instead)
MAE: Simple but doesn’t punish bad predictions as obviously as MSE
RMSE: Addresses the issue of earnings being in a large number amount but more difficult to interpret

Given that our model provides reliable information when predicting gross box office earnings, the hope is for our model to help movie studios, producers, and investors to make more informed business decisions about budgeting, marketing, and distribution of their movies. Alternatively, our analysis may reveal that the features we used are not informative enough, even given a variety of complex models and approaches, in which we would determine that the data obtained from IMDb should not be used alone in future research and may need to include more complex features or deeper additional analysis such as sentiment analysis of reviews. Additionally, if we determine that our models’ accuracy stagnate even as we add more complexity (accounting for methods to prevent overfitting), we may conclude that the features we did use poorly map onto box office earnings, suggesting that IMDb data is wholly uninformative when determining the financial success of a movie. As stated in the problem statement, we will likely explore multiple kinds of losses/error metrics, mostly because individually each metric can be both good and bad based on our data. In the first reference (“Predicting Movie Revenue from IMDb Data”), MAPE is particularly noted to be a poor metric because of the issue of lower earning movies having greater error on worse mispredictions [1]. Their data, however, only looks at 4052 movies whereas we have over 18000, meaning there is a possibility that their analysis may be not entirely informed.


# Data
Link to dataset: <a>https://www.kaggle.com/datasets/rajugc/imdb-movies-dataset-based-on-genre?resource=download</a>

Column Descriptions:

 • movie_id - IMDB Movie ID

 • movie_name - Name of the movie

 • year - Release year

 • certificate - Certificate of the movie

 • run_time - Total movie run time

 • genre - Genre of the movie

 • rating - Rating of the movie

 • description - Description of the movie

 • director - Director of the movie

 • director_id - IMDB id of the director

 • star - Star of the movie

 • star_id - IMDB id of the star

 • votes - Number of votes in IMDB website
 
 • gross(in $) - Gross Box Office of the movie


The dataset starts with 298975 total observations, and 14 variables. Once we remove the observations with Null values, we have 18709 total observations. Additionally, we are only interested in 6 of the variables as aforementioned because several of the variables are string values that would be too lengthy to one-hot encode. A valid observation must include the features: year (integer), certificate, runtime (integer), genre, rating (float value),  votes (integer), and gross income (float), and it must also have no null values. These variables are a combination of ordinal variables and categorical variables which are able to be one-hot encoded. Therefore they will be usable by our program.

Many of the movies in the dataset are listed under several genres. In total there are 40 unique genres listed in the dataset, but the data is organized into 13 folders: one for each main genre of movie. To save some computational expense in feature selection, we are going to only use the genre that corresponds to the directory that the movie was initially listed under. For the same reason, we will chunk the year data into groups of 15 years. 

The variables that we will one-hot encode will include genre and certificate. There are 27 unique certificates and 13 unique genres in our dataset that we are interested in (action, adventure, crime, family, fantasy, film-noir, history, horror, mystery, scifi, sports, thriller, and war). We can also one-hot encode year by first grouping movies into a new feature which will represent a range of years (e.g. 1975-1999 movies) and placing movies into that range if their year is within it, and one-hot encoding each entry of the list of ranges (e.g. 1950-1974, 1975-1999, 2000-2022).

We are going to eliminate all outliers from the year, runtime, rating, votes, and gross income categories. In order to do so we will discard all dataset outside of 1.5*IQR above and below the upper and lower quartiles. 

We will check for multicollinearity using a correlation matrix. We will eliminate values with correlation coefficients greater than 0.7.

# Proposed Solution

We are going to use a few algorithms to approach the solution to this problem, but they are subsets of linear and nonlinear regression. This can be achieved through the use of sklearn's linear_model.LinearRegression (and its relatives Ridge and LASSO regression). In addition to this, we will utilize a decision tree model as well and utilize the machine learning model that scores better on our performance metrics. Our problem is clearly geared towards a regression solution as we aim to predict gross earnings from our features, instead of a classification problem in which logistic regression would be preferred. To test our solution, we will run cross-validation on our dataset to generate an estimate of our model's performance. A possible benchmark is simply guessing the mean gross box office earnings.


Because our problem has a very simple goal to achieve, we want to introduce nuance via complex approaches to analyzing the algorithms we use. We’ll use a few different standard model approaches including Gradient Boosting and Random Forest, as well as more complex approaches such as XGBoost and compare the results across each model. Again, because of the simplicity of our problem, we can also extend our analysis to using some hyperparameter optimization techniques, but since models like XGBoost get exceedingly complicated in the number of hyperparameters, we will likely choose instead to do random search and hope to establish good baselines for each approach, especially since we are most likely to use stratified k-fold cross-validation in order to preserve (for example) genre ratio while having a reasonably sized test set (given that our dataset has over 18000 examples). 

# Evaluation Metrics

To predict the box office gross income of movies based on their year, certificate, runtime, genre, rating and total number of votes, we will be potentially using the following metrics. 

Mean Absolute Error to measure the average absolute difference between the predicted box office gross and the actual box office gross across all the movies.

MAE = (1 / n) * sum(abs(y_true - y_pred))

Mean Squared Error and Root Mean Squared Error to measure the square root of the average squared difference between the predicted box office gross and the actual box office gross.

MSE = (1 / n) * sum((y_true - y_pred)^2)

RMSE = sqrt((1 / n) * sum((y_true - y_pred)^2))

And R2 Score to measure how well the regression model fits the actual data

R2 Score = 1 - (sum((y_true - y_pred)^2) / sum((y_true - mean(y_true))^2))

y_true = actual box office gross

y_pred = predicted box office gross

# Preliminary results

We have created our boxplots to rule out any outliers. 

Our data has more than 2000 outliers, this is almost 20% of our data. So, we will have to go through manually and pick out which ones we truly need to discard and which ones are a little less necessary. 

We computed a correlation matrix for our numerical data and determined that with a threshold of 0.7 we do not have any multicollinearity. 




# Ethics & Privacy

The IMDB dataset is downloaded from the Kaggle website with no personal nor sensitive information included from human participants. All the variables in the dataset are available to public on IMDB website in which we can eliminate the concern for data privacy and misuse of the data. The movie ratings and box office gross are largely influenced by the viewers and fans of particular franchises and movie stars which may be biased, however, since we are using a large dataset with a huge number of ratings, the bias could be neglected or will be evaluated and addressed if the concerns were to arise. Our final product will be available to students of current COGS118A class, future prospective students and potentially to public and we will carefully monitor and identify the unintended use of our project.

# Team Expectations 

* Communicate through Discord about upcoming deadlines and set up zoom meetings as announcements
* If struggling with a particular problem, consult other members about possible solutions/alternate approaches
* In the event of roadblocks, ping members responsible for the roadblock to get a timeline for roadblock
* Weekly meet-in (TBD) to get status of members (pushes, roadblocks, brainstorming)
* Prior to deadlines have a quick meet to see if everyone has pushed their changes


# Project Timeline Proposal


| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 2/22  |  7 PM | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | Complete Project Proposal  |
| 2/29  |  5 PM |  Outline Code and what we need to do for the Checkpoint | Get started building project and working on Checkpoint; assign specific tasks to groupmates. | 
| 3/3  | 5 PM  | Fine tune checkpoint and work on individual components | Go over and submit checkpoint; continue working on project   |
| 3/10  | 6 PM  | Complete/fine tune background, abstract, and EDA | Review/Edit wrangling/EDA; Discuss Analysis Plan; Continue working on code |
| 3/17  | 12 PM  | Finalize programming for project | Begin compiling everything and making it all cohesive.
| 3/20  | 12 PM  | Work on any edits that we have established we need | Discuss/edit full project |
| 3/22  | Before 11:59 PM  | NA | Turn in Final Project  |

# Footnotes
<a name="yoonote"></a>1.[^](#yoo): Yoo, Steven, et al. “Predicting Movie Revenue from IMDb Data.” CiteSeerX, https://citeseerx.ist.psu.edu/doc_view/pid/6e6cdf5b0282d89de45c407fc76a4c218696e3e3. pdf/82778929.pdf. <br> 

<a name="dhirnote"></a>2.[^](#dhir): Dhir R, Raj A (2018) Movie success prediction using machine learning algorithms and their comparison. In: 2018 First International Conference on Secure Cyber Computing and Communication (ICSCCC), IEEE, pp 385–390. <br>

<a name='abidinote'></a>3.[^](#abidi): Abidi, S.M.R., Xu, Y., Ni, J. et al. Popularity prediction of movies: from statistical modeling to machine learning techniques. Multimed Tools Appl 79, 35583–35617 (2020). https://doi.org/10.1007/s11042-019-08546-5. <br>